In [1]:
import numpy as np
import pandas as pd
from glob import glob
from functools import reduce

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
# 크롤링한 데이터 로드
def load_data(source):

    # 한강에 있는 대교들의 수위 및 유량------------------
    if(source=='bridge'):
        jamsu = sorted(glob("../data/bridge/서울시(잠수교)/*.csv"))
        cheongdam =sorted(glob("../data/bridge/서울시(청담대교)/*.csv"))
        haengju = sorted(glob("../data/bridge/서울시(행주대교)/*.csv"))
        hangang = sorted(glob("../data/bridge/서울시(한강대교)/*.csv"))
        gwangin = sorted(glob("../data/bridge/서울시(광진교)/*.csv"))
        paldang = sorted(glob("../data/bridge/남양주시(팔당대교)/*.csv"))
        jungnang = sorted(glob("../data/bridge/서울시(중랑교)/*.csv"))
        
        data_list={"jamsu":jamsu, "cheongdam":cheongdam, "haengju":haengju, 
                   "hangang":hangang, "gwangin":gwangin, "paldang":paldang, "jungnang":jungnang}
        drop_cols=['wlobscd', 'links']
        
    # 팔당댐 관련 변수들들 ----------------------------------------
    elif(source=='dam'):
        pd_dam = sorted(glob("../data/dam/팔당댐/*.csv"))
        
        data_list={'pd_dam':pd_dam}
        drop_cols=['dmobscd','links']
        
    # 강화대교 조위-----------------------------------
    elif(source=='tide'):
        tide_lv = sorted(glob("../data/tide/강화대교/*.csv"))
        
        data_list={'tide_lv':tide_lv}
        drop_cols=[]
    # 강수량 ------------------------------------------
    elif(source=='rf'):
        zingwan = sorted(glob("../data/rf/남양주시(진관교)/*.csv"))
        daegog = sorted(glob("../data/rf/서울시(대곡교)/*.csv"))
        songjeong = sorted(glob("../data/rf/서울시(송정동)/*.csv"))
        
        data_list={'zingwan':zingwan, 'daegog':daegog, 'songjeong':songjeong}
        drop_cols=['rfobscd','links']
        
    return data_list,drop_cols

In [3]:
def preprocessing(source,data_storage):
    
    # 수위값을 해발표고 값으로 변환
    # 한강홍수통제소에서 수위<->해발표고의 차이를 비교하여 개별 튜닝해야 함.
    if(source=='bridge'):
        calc={"jamsu":-6.8, "cheongdam":174.7, "haengju":80.3, 
            "hangang":207, "gwangin":528, "paldang":557.0, 
            "jungnang":917}
        
        for i in data_storage:
            data_storage[i]['wl']=data_storage[i]['wl'].replace(' ','0')
            data_storage[i]['wl']=data_storage[i]['wl'].astype('float')
            data_storage[i]['wl']=data_storage[i]['wl']*100 +calc[i]
            
            data_storage[i].columns=['ymdhm','wl_'+i,'fw_'+i]
            
    elif(source=='rf'):
        for i in data_storage:
            data_storage[i].columns=['ymdhm','rf_'+i]
        
    return data_storage

In [4]:
def save_data(source,data_storage):
    
    # 데이터들을 병합하고 저장 총 4개(bridge, dam, rf, tide) 데이터가 생성됨
    data=[data_storage[i] for i in data_storage]
    data = reduce(lambda left, right: pd.merge(left, right, on='ymdhm'), data)
    data.replace(" ",np.nan,inplace=True)
    data = data.sort_values(by='ymdhm', ascending=True)
    data = data.drop_duplicates(subset='ymdhm', keep='first')
    data.to_csv(f'../data/{source}/full_{source}.csv')
    
    return data
    

In [5]:
def merge_data(source):
    
    data,drop_cols=load_data(source)
    # 각 대교의 이름과 데이터들이 담겨있는 리스트 
    data_storage=dict()
    
    for river in data.keys():
        print("#"*20)
        print(f"{river} data Merge Start")
        tmp_df = []
        # 년,월별로 나뉘어진 데이터들을 병합
        for file in data[river]: 
            tmp = pd.read_csv(file)
            tmp_df.append(tmp)
        full_df = pd.concat(tmp_df, axis=0)
        full_df.reset_index(drop=True, inplace=True)
        full_df.drop(drop_cols, axis=1, inplace=True)
        if source!='tide':full_df['ymdhm'] = pd.to_datetime(full_df['ymdhm'], format='%Y%m%d%H%M')
        # 조위 데이터의 경우 형식이 달라 개별 처리
        else: 
            full_df['record_time'] = pd.to_datetime(full_df['record_time'], format='%Y-%m-%d %H:%M:%S')
            full_df.columns=['ymdhm','tide_level']
        full_df=full_df.sort_values(by='ymdhm')
        full_df=full_df.reset_index(drop=True)
        data_storage[river]=full_df
        print(f"{river} data Merge complete")
    return data_storage


In [8]:
def check_data():
    prev = glob("../data/full_data.csv")
    print(prev)
    if (prev != []):
        return 1
    else:
        return 0

In [9]:
def finalize_data():
    answer = int(input("파일을 병합하려면 1, 아니면 0을 입력하세요. "))
    if (answer == 1):
        print(check_data())
        if (check_data()) == 0:
            dam_data = pd.read_csv("../data/dam/full_dam.csv", index_col=0)
            bridge_data = pd.read_csv(
                "../data/bridge/full_bridge.csv", index_col=0)
            # dam_data = pd.read_csv("../data/dam/full_dam.csv", index_col=0)
            rf_data = pd.read_csv("../data/rf/full_rf.csv", index_col=0)
            tide_data = pd.read_csv("../data/tide/full_tide.csv", index_col=0)
            final_df = [bridge_data, dam_data, rf_data, tide_data]
            final_data = reduce(lambda left, right: pd.merge(
                left, right, on='ymdhm', how='outer'), final_df)
            final_data = final_data[:-1]
        else:
            print("이미 데이터가 존재합니다.")
            final_data = pd.read_csv("../data/full_data.csv", index_col=0)
    else:
        print("작업을 종료합니다.")
        return 0
    return final_data

In [11]:
while True:
    source = input(
        "bridge / dam / rf / tide 중 하나를 입력하세요.(종료하려면 아무 키나 입력하세요): ")
    if source not in ['bridge', 'dam', 'rf', 'tide']:
        break
    if (check_data() == 0):
        data_storage = merge_data(source)
        data_storage = preprocessing(source, data_storage)
        data = save_data(source, data_storage)
    else:
        break
    print("요청하신 데이터 전처리 및 저장이 완료되었습니다.")
final_data = finalize_data()

['../data/full_data.csv']
['../data/full_data.csv']
1
['../data/full_data.csv']
이미 데이터가 존재합니다.


C:\Users\Matthew624.DESKTOP\AppData\Local\Temp\ipykernel_18668\3206401129.py:18: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  final_data = pd.read_csv("../data/full_data.csv", index_col=0)
